In [1]:
import pandas as pd
import numpy as np
import csv
import os

### CRF genes in complexes: annotation

In [2]:
go_use_file = 'https://raw.githubusercontent.com/ScrippsPipkinLab/CRF_Screen/master/Ref/GO_terms/complex_count_rank_select_for_plotting.csv'
go_use_df = pd.read_csv(go_use_file)

crf_go_file = 'https://raw.githubusercontent.com/ScrippsPipkinLab/CRF_Screen/master/Ref/GO_terms/CRM_complexes_count.csv'
crf_go_df = pd.read_csv(crf_go_file)

crf_anno_file = 'https://raw.githubusercontent.com/ScrippsPipkinLab/CRF_Screen/master/Ref/HGSCore_only-CRF_anno.csv'
crf_anno_df = pd.read_csv(crf_anno_file)

*Find alternative gene names*

In [3]:
crf_alt_names_file = 'https://raw.githubusercontent.com/ScrippsPipkinLab/CRF_Screen/master/Ref/CRF_alternative_gn.csv'
crf_alt_names_df = pd.read_csv(crf_alt_names_file)
alt_names_dict = {crf_alt_names_df['gene_name'][i]:crf_alt_names_df['Alternative'][i] 
                  for i in range(len(crf_alt_names_df))}
alt_names_dict_rev = {crf_alt_names_df['Alternative'][i]:crf_alt_names_df['gene_name'][i] 
                  for i in range(len(crf_alt_names_df))}

def alt_genes(in_list):
    out_list = []
    for i in in_list:
        if i in alt_names_dict.keys():
            out_list.append(alt_names_dict[i])
        elif i in alt_names_dict_rev.keys():
            out_list.append(alt_names_dict_rev[i])
    return(out_list)

## Other crfs to be annotated
- Chd

In [4]:
all_complexes_dict = {}

In [5]:
uniq_complexes = [x for x in np.unique(crf_anno_df['complexNames'].tolist()) if str(x) != 'nan']

for i in uniq_complexes:
    i_crfs = crf_anno_df[crf_anno_df['complexNames'] == i]['gene_name'].tolist()
    i_crfs = [x.capitalize() for x in i_crfs]
    i_crfs += alt_genes(i_crfs)
    all_complexes_dict[i] = i_crfs

In [6]:
go_use_terms = go_use_df[go_use_df['use'] == 'yes']['complex'].tolist()

for i in go_use_terms:
    i_crfs = crf_go_df[crf_go_df[i] == 'Yes']['gene_name'].tolist()
    i_crfs = [x.capitalize() for x in i_crfs]
    i_crfs += alt_genes(i_crfs)
    all_complexes_dict[i] = i_crfs

In [10]:
all_complexes_dict['Chd'] = [x for x in crf_go_df['gene_name'] if 'Chd' in x]

In [11]:
out_file = 'CRF_complexes.csv'
with open(out_file, "w") as fout:
    wfout = csv.writer(fout, delimiter=",")
    wfout.writerow(['complex', 'gene_name'])
    
    for i in all_complexes_dict.keys():
        i_genes = all_complexes_dict[i]
        for g in i_genes:
            wfout.writerow([i, g])